In [1]:
# If this ipynb file is called from another notebook, skip this block...
import os 
import sys
from dotenv import load_dotenv

if not os.getcwd().endswith('scripts'):
    os.chdir('scripts')
    sys.path.append('..')

load_dotenv('../../../.envrc')

True

In [2]:
import openai 
import os
import json 
import yaml

# Set the openai.api_key from environment variable: OPENAI_API_KEY
openai.api_key = os.environ["OPENAI_API_KEY"]

with open('../categories.yaml', 'r') as f:
    categories = yaml.safe_load(f)

# Lowercase all keywords for accurate matching
for category, keywords in categories.items():
    categories[category] = [keyword.lower() for keyword in keywords]

category_names = list(categories.keys())
category_names

['Charts and Diagrams',
 'Chat Bots and Assistants',
 'Coding',
 'Documents',
 'Earth and Space',
 'Education and Learning',
 'Entertainment',
 'Events',
 'Finance',
 'Food and Drink',
 'Health and Fitness',
 'Images',
 'Job and Career',
 'Legal and Politics',
 'Marketing',
 'News and Media',
 'Productivity',
 'Prompts',
 'QR Codes',
 'Real Estate',
 'Search',
 'Shopping',
 'Social Networking',
 'Sports',
 'Tabular Data',
 'Text',
 'Travel',
 'Uncategorized',
 'Vehicles',
 'Video']

In [3]:
import re

# Define a function to match the category keywords and extract the category name
def extract_category(message, categories):
    # Split the message into lines and get the last line
    category_line = message.strip().split('\n')[-1]

    # Lowercase the category line for accurate matching
    category_line = category_line.lower()

    # Loop over all the categories and their keywords
    for category, keywords in categories.items():
        # Lowercase all the keywords for accurate matching
        keywords = [keyword.lower() for keyword in keywords]

        # Check if any of the keywords match the category line
        if any(keyword in category_line for keyword in keywords):
            return category

    # If no category is found, return None
    return None

# Example usage
message = "Plugin: My Plugin\nDescription: This is a plugin for doing things.\nCategory: productivity. Is there anything else you'd like help with?"
category = extract_category(message, categories)
print(category)

Productivity


In [4]:
with open("chatgpt_plugins.json", 'r') as f:
    plugin_file = json.load(f)

if os.path.exists("chatgpt_plugin_descriptions.json"):
    with open("chatgpt_plugin_descriptions.json", 'r') as f:
        plugin_descriptions = json.load(f)
else:
    plugin_descriptions = {"items": []}

# Count the number of plugins in plugin_file:
num_plugins = len(plugin_file["items"])
print(f"Found {num_plugins} plugins in the plugins file.")

# Count the number of plugins in plugin_descriptions:
num_descriptions = len(plugin_descriptions["items"])
print(f"Found {num_descriptions} plugins in the descriptions file.")


Found 1000 plugins in the plugins file.
Found 990 plugins in the descriptions file.


In [5]:
# Print all of the plugin names that are in plugin_descriptions["items"] but not in plugin_file["items"], by manifest.name_for_model
for plugin in plugin_descriptions["items"]:
    if plugin["manifest"]["name_for_model"] not in [plugin["manifest"]["name_for_model"] for plugin in plugin_file["items"]]:
        print(f"Plugin {plugin['manifest']['name_for_model']} in plugin descriptions is not found in plugin_file.")
        # Remove the plugin from plugin_descriptions
        plugin_descriptions["items"].remove(plugin)
        

Plugin word_sneak in plugin descriptions is not found in plugin_file.
Plugin Ambition in plugin descriptions is not found in plugin_file.
Plugin AppyPieAIAppBuilder in plugin descriptions is not found in plugin_file.
Plugin tutory in plugin descriptions is not found in plugin_file.
Plugin speechki_tts_plugin in plugin descriptions is not found in plugin_file.
Plugin magic in plugin descriptions is not found in plugin_file.
Plugin MixerBox_FreecableTV in plugin descriptions is not found in plugin_file.
Plugin Shop in plugin descriptions is not found in plugin_file.
Plugin expedia in plugin descriptions is not found in plugin_file.
Plugin woxo in plugin descriptions is not found in plugin_file.


In [6]:
import time
import openai

gpt_model = "gpt-4-1106-preview"

# Define function to send chat request
def send_chat_request(system_message, user_message, max_retries=5):
    retries = 0 
    while retries < max_retries:
        try:
            response = openai.ChatCompletion.create(
                model=gpt_model,
                temperature=0.5,
                max_tokens=600,
                messages=[system_message, user_message]
            )
            return response
        except openai.error.Timeout as e:
            #Handle timeout error, e.g. retry or log
            print(f"OpenAI API request timed out: {e}")
        except openai.error.APIError as e:
            #Handle API error, e.g. retry or log
            print(f"OpenAI API returned an API Error: {e}")
        except openai.error.APIConnectionError as e:
            #Handle connection error, e.g. check network or log
            print(f"OpenAI API request failed to connect: {e}")
        except openai.error.InvalidRequestError as e:
            #Handle invalid request error, e.g. validate parameters or log
            print(f"OpenAI API request was invalid: {e}")
        except openai.error.AuthenticationError as e:
            #Handle authentication error, e.g. check credentials or log
            print(f"OpenAI API request was not authorized: {e}")
        except openai.error.PermissionError as e:
            #Handle permission error, e.g. check scope or log
            print(f"OpenAI API request was not permitted: {e}")
        except openai.error.RateLimitError as e:
            #Handle rate limit error, e.g. wait or log
            print(f"OpenAI API request exceeded rate limit: {e}")
        except openai.error.ServiceUnavailableError as err:
            print("Server error, retrying in 5 seconds...")
            time.sleep(5) # wait for 5 seconds before retrying
        retries += 1
    raise Exception(f"Maximum number of retries({max_retries}) exceeded")

In [7]:
import json
import re

current_version = 1


# Loop over all the plugins in the file
# for plugin in plugin_file["items"]:
for i in range(0, len(plugin_file["items"]), 10):
    plugins = plugin_file["items"][i:i+10]

    for plugin in plugins:
        # Get the name_for_human and description_for_model fields from the plugin
        name = plugin["manifest"]["name_for_human"]
        plugin_name = plugin['manifest']['name_for_model']
        plugin_id = plugin['id']
        human_description = plugin['manifest']['description_for_human']
        full_description = plugin["manifest"]["description_for_model"]

        # Check if the plugin_id already exists in plugin_descriptions
        if any(p['id'] == plugin_id for p in plugin_descriptions["items"]):
            print(f"Plugin {plugin_name} already exists")

            # Find the existing plugin with the matching id
            existing_plugin = next(p for p in plugin_descriptions["items"] if p['id'] == plugin_id)

            # Check if we need to re-do the query if the plugin was generated with an older version of the model
            if existing_plugin['manifest'].get('generated_version', 0) < current_version:
                print(f"Generated version out of date. Re-running plugin {plugin_name}")
            else:
                print(f"Skipping plugin {plugin_name}")
                continue

        # Create a system message to instruct the model to generate a description for the plugin
        system_message = {
            "role": "system",
            "content": f"Your task is to rewrite the description for the plugin. The new description should be useful for a user to understand how to use the plugin, it should be in a factual and objective tone, and it should accurately depict the plugin's functionality based on the original description, and the model's description for the plugin. The description should avoid promotional language and stick to the essential functions of the plugin. After creating the description, do your best to classify the plugin into one of the following categories: {category_names}. The new description should be a concise single paragraph without any links. Please follow the format: 'Description: <description>\n\nCategory: <category>'"
        }


        # Create a user message to provide the name and description fields as input
        user_message = {
            "role": "user",
            "content": f"Plugin: {name};\nOriginal Description: {human_description}; Model instructions to use as reference: {full_description}; \nCategory: "
        }

        # Send a request to the chat completions API endpoint with the GPT3.5-turbo model and the messages parameter
        response = send_chat_request(system_message, user_message)
        message = response['choices'][0]['message']['content']

        # message will contain "Plugin: {plugin name}\nDescription: {description}\nCategory: {category}"
        category = extract_category(message, categories)

        # Find the description. It will start with: 'Description:' and end with '\n\nCategory'
        # description = re.search(r'Description:(.*)\n\nCategory', message, re.DOTALL).group(1).strip()
        match = re.search(r'Description:(.*)\nCategory', message, re.DOTALL)
        if match:
            description = match.group(1).strip()
        else:
            print(f"Error: Bad description for plugin {plugin_name}")
            print(response)
            continue

        print(f"{name}; {category}; {description}\n")

        # Add generated_description into the plugin['manifest'] and append the plugin to the plugin_descriptions.json
        plugin['manifest']['generated_description'] = description
        plugin['manifest']['category'] = category
        plugin['manifest']['generated_version'] = current_version

        # If the plugin already exists in plugin_descriptions['items'], let's overwrite it. Otherwise append
        if any(p['id'] == plugin_id for p in plugin_descriptions["items"]):
            for p in plugin_descriptions['items']:
                if p['id'] == plugin_id:
                    p = plugin
        else:
            plugin_descriptions['items'].append(plugin)

        # Write the plugin_descriptions to the file every 10 plugins
        if len(plugin_descriptions['items']) % 10 == 0:
            with open('chatgpt_plugin_descriptions.json', 'w') as f:
                json.dump(plugin_descriptions, f, indent=4)

# Write the remaining plugin descriptions to the file
with open('chatgpt_plugin_descriptions.json', 'w') as f:
    json.dump(plugin_descriptions, f, indent=4)



Plugin Sesotec already exists
Skipping plugin Sesotec
Plugin autolab_causality already exists
Skipping plugin autolab_causality
Pictory.ai; Video; Pictory.ai is a plugin designed to transform written scripts into engaging videos. Users input their text, and the plugin generates a video based on the provided content. If the video is not available instantly, the plugin will continue to check at 15-second intervals until the video URL is produced, which users can then access.

Aardvark News; News and Media; Aardvark News is a plugin designed to provide users with the latest news stories from a vast array of over 80,000 global news sources. Users can request news updates on various topics, including specific countries, trending issues, or particular subjects like technology or politics. The plugin allows for generation of tweets and news digests, and it presents information in markdown format, including links and images when available. Users should prompt the plugin with specific requests 

KeyboardInterrupt: 